In [ ]:
import os
from avapi.carla import CarlaScenesManager


cpath = os.path.join("/data/shared/CARLA/multi-agent-v1/")
# cpath = "../examples/sim_results"
CSM = CarlaScenesManager(cpath)
print(CSM.scenes)
CDM = CSM.get_scene_dataset_by_name(CSM.splits_scenes["val"][0])
print(f"{len(CDM)} frames")

In [ ]:
from avstack.modules.perception.object2dfv import MMDetObjectDetector2D
from avstack.modules.perception.object3d import MMDetObjectDetector3D


# init detectors|
detector_2d =  MMDetObjectDetector2D(model="fasterrcnn", dataset="carla-joint", threshold=0.5)
detector_3d = MMDetObjectDetector3D(model="pointpillars", dataset="carla-vehicle", threshold=0.5)
detector_3d_inf = MMDetObjectDetector3D(model="pointpillars", dataset="carla-infrastructure", threshold=0.2, epoch=2)

In [ ]:
from avapi.visualize.snapshot import show_lidar_bev_with_boxes, show_lidar_on_image, show_image_with_boxes

# load data
agent = 5
frame_idx = 40
camera_sensor = "camera-0"
lidar_sensor = "lidar-0"
frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)

# 2d detector
objs_2d = detector_2d(img)
if agent < 5:
    objs_3d = detector_3d(pc)
else:
    objs_3d = detector_3d_inf(pc)

# show_lidar_bev_with_boxes(pc=pc, boxes=objs)
# show_lidar_on_image(img=img, pc=pc, boxes=objs)
# show_image_with_boxes(img=img, boxes=objs, inline=True, return_image=False)
# ref_gp = pc.calibration.reference.get_ground_projected_reference()
# for obj in objs_3d:
#     box = obj.box.change_reference(ref_gp, inplace=False)
#     print(box.position.x[2])
# print([obj.box.position.x[2] for obj in objs_3d])

show_lidar_bev_with_boxes(pc=pc, boxes=objs)
show_lidar_bev_with_boxes(pc=pc, boxes=objs_3d)
show_image_with_boxes(img=img, boxes=objs_3d, inline=True, return_image=False)


## Test projection operation

In [ ]:
# load data
agent = 6
frame_idx = 5
camera_sensor = "camera-0"
lidar_sensor = "lidar-0"
frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)


In [ ]:
import numpy as np
from avstack.calibration import LidarCalibration
from avstack.geometry import transform_orientation

# calib_new = LidarCalibration(reference=pc.reference.get_ground_projected_reference())
# pc_new = pc.project(calib_new)
pc_new = pc.transform_to_ground()
boxes_new = [obj.box.change_reference(pc_new.reference, inplace=False) for obj in objs]

print(transform_orientation(pc.reference.reference.q, 'quat', 'euler') * 180/np.pi)
print(transform_orientation(pc_new.reference.q, 'quat', 'euler') * 180/np.pi)
print([box.position.x[2] for box in boxes_new])

In [ ]:
pc_new.data.x.max(axis=0)

In [ ]:
pc_new.data.x.min(axis=0)

In [ ]:
from avstack.geometry import PointMatrix3D
from avstack.sensors import LidarData
from avapi.carla.dataset import read_objects_from_file, read_pc_from_file, read_calibration_from_file


# load in data from projected
lidar_filepath = (
    CDM.get_sensor_file(
        frame, timestamp=None, sensor=lidar_sensor, agent=agent, file_type="data"
    )
    + ".bin").replace(f"data/{lidar_sensor}", f"data-projected/{lidar_sensor}")

calib_filepath = (
    CDM.get_sensor_file(
        frame, timestamp=None, sensor=lidar_sensor, agent=agent, file_type="calib"
    )
    + ".txt"
)
calib = read_calibration_from_file(calib_filepath)
calib.reference = calib.reference.get_ground_projected_reference()

# format properly
pc_data = PointMatrix3D(
    x=read_pc_from_file(lidar_filepath, n_features=4, filter_front=False),
    calibration=calib,
)
pc_new = LidarData(
    data=pc_data,
    calibration=calib,
    timestamp=0,
    frame=frame,
    source_ID=lidar_sensor,
)

In [ ]:
calib.reference.reference

In [ ]:
pc_new.data.x.min(axis=0)

In [ ]:
pc_new.data.x.max(axis=0)

In [ ]:
show_lidar_bev_with_boxes(pc=pc, boxes=objs)
show_lidar_bev_with_boxes(pc=pc_new, boxes=boxes_new)

In [ ]:
np.min(pc.data.x, axis=0)
np.max(pc.data.x, axis=0)